# EMPTYNESS AND FINITENESS PROBLEM OF CFG

In [1]:
import re
from collections import deque,Counter
import networkx

class notValidCFG(Exception):
    def __init__(self,msg):
        super().__init__(msg)

class CFG:
    """
    CFG -> CONTEXT FREE GRAMMER
    ----------
    DEFINES THE RULES TO GENERATE THE STRINGS FROM SET OF SYMBOLS
    
    It is Defined using 4-tuple G = {V,T,S,P} where V is set of Variables, T is set of Terminals, S is Starting Symbol (S ⊂ V), P is Production Symbol

    The Variables must be in Uppercase and Terminals must be in Lowercase

    Start Symbol should be S

    Add productions line by line in string format
    Write Multiple Productions Seperated by | or /
    Ex: S->A|bB/C

    write ε for null transition
    """
    variables : set
    terminals : set
    start_symbol : chr
    productions : dict
    originalRules : set
    EPSILON = "ε" 

    # if cfg is not valid then raises error
    def __init__ (self,productions):

        self.variables = set()
        self.terminals = set()
        self.start_symbol = "S"
        self.productions = dict()  # map -> key = NT, Value = (NT T)*
        self.originalRules = set() # original production rules in string format

        productions = productions.strip()
        for i in productions.split():
            left,right = i.split('->')
            left = left.strip()
            if(len(left)!=1):
                raise notValidCFG("CFG should be of format α -> β, where len(α)==1")
            if(not left.isupper()):
                raise notValidCFG("α -> β, α should be non-terminal symbol")
            right = [j.strip() for j in re.split("\/|\|",right)]

            if(left not in self.productions):
                self.productions[left] = []

            for val in right:
                self.productions[left].append(val)
                for k in val:
                    if(k.isupper()):
                        self.variables.add(k)
                    elif(k == self.EPSILON or k.islower() or k.isdigit()):
                        self.terminals.add(k)

            self.variables.add(left)
        
        if('S' not in self.variables):
            raise notValidCFG("Starting Symbol 'S' Missing")
        
        rules = set()
        for i in self.productions:
            for j in self.productions[i]:
                rules.add(f"{i}->{j}")
        self.originalRules = rules

    def __str__(self) -> str:
        
        rules = set()

        for i in self.productions:
            for j in self.productions[i]:
                rules.add(f"{i}->{j}")

        s = "CFG (V,T,S,P) = {\n" + f"\tV = {self.variables}\n\tT = {self.terminals}\n\tS = '{self.start_symbol}'\n\tP = {rules}"+"\n}"
        return s
    
    def reachableFromStart(self): # uses bfs approach
        
        """It finds all Non Terminal Symbols reachable from Start 'S'"""
        
        reachable = set()

        queue = deque()
        queue.append('S')

        while(len(queue)>0):
            symbol = queue.popleft()
            reachable.add(symbol)
            if(symbol in self.productions):
                for prod in self.productions[symbol]:
                    for token in prod:
                        if(token not in reachable and token in self.variables):
                            queue.append(token)
        return reachable

    def removeUnreachable(self,reachable):

        """ Remove All Non Terminals that are non reachable from Start S """

        nonreachable = self.variables - reachable
        for i in nonreachable:
            if(i in self.productions):
                del self.productions[i]
        self.variables = reachable

    def deriveTerminalSymbol(self):
        """ Finds Non terminal which can derive the terminal string """
        
        productive = {i:False for i in self.variables}
        # whenever we find a production which generates terminal string on right side, we mark it as true

        sum = -1
        while(sum!=0):  # continue until no changes can be made
            sum = 0
            for prod in self.productions:
                if(productive[prod]==False):
                    for right in self.productions[prod]:
                        if(all(i in self.terminals or productive[i] for i in right)):
                            sum+=1
                            productive[prod] = True
                            break
        return {i for i in productive if productive[i]}
                
    def removeProductive(self,productive):
        
        """ Removes all Non Terminals which cannot derive the terminal string (results in formation of loop among variables) """
        non_productive = self.variables - productive

        new_rules = {}

        for prod in self.productions:
            if(prod in productive):
                new_rules[prod] = []
                for right in self.productions[prod]:
                    if(all(i not in non_productive for i in right)):
                        new_rules[prod].append(right)
                    
        self.productions = new_rules
        self.variables = productive

    def remove_useless_production(self):
        
        # Removes Non terminal which are not reachable from start
        reachable = self.reachableFromStart()
        self.removeUnreachable(reachable)

        # Removes Non terminal which do not produce terminal symbol
        productive = self.deriveTerminalSymbol()
        self.removeProductive(productive)


    def remove_unit_production(self):
        
        """ Remove Productions of form A->B, where both are Non-Terminal Symbols """

        # finding unit productions
        unit_prod = dict()
        for prod in self.productions:
            for j in self.productions[prod]:
                if(len(j)==1 and j in self.variables):  # form : A->B 
                    if(prod not in unit_prod):
                        unit_prod[prod] = []
                    unit_prod[prod].append(j)

        # finding closure of every unit production symbol

        for prod in unit_prod:
            closure = set()
            q = deque()
            for j in unit_prod[prod]:
                q.append(j)
            
            while(len(q)>0):
                nt = q.popleft()
                closure.add(nt)
                if(nt in unit_prod):
                    for k in unit_prod[nt]:
                        q.append(k)
            
            unit_prod[prod] = closure

        # removing unit production using closure
        
        remove_units = set()
        for prod in unit_prod:
            for j in unit_prod[prod]:
                if(j in unit_prod and j not in remove_units):
                    remove_units.add(j)
        
        update_unit_prod = dict()
        for prod in unit_prod:
            update_unit_prod[prod] = []
            for j in unit_prod[prod]:
                if(j not in remove_units and j in self.productions):
                    update_unit_prod[prod].extend(self.productions[j])

        new_rules = dict()
        for prod in self.productions:
            new_rules[prod] = []
            for choice in self.productions[prod]:
                if(len(choice) == 1 and choice in update_unit_prod):
                    new_rules[prod].extend(update_unit_prod[choice])
                elif(len(choice) == 1 and choice in self.productions and len(self.productions[choice])==1):
                    new_rules[prod].extend(self.productions[choice])
                else:
                    new_rules[prod].append(choice)

        self.productions = new_rules

    def remove_null_production(self):
        
        """ It removes ε symbol from all productions """

        # first finding all non-terminals which generate epsilon symbols
        null_var = set()
        for prod in self.productions:
            if('ε' in self.productions[prod]):
                null_var.add(prod)
        
        def powerset(s):
            if len(s) == 0:
                return [[]]
            else:
                result = []
                for subset in powerset(s[1:]):
                    result.append(subset)
                    result.append([s[0]]+subset)
                return result
        
        new_rules = dict()

        for prod in self.productions:
            new_rules[prod] = []
            for right in self.productions[prod]:
                if(right=='ε'):
                    continue
                var = any([i in null_var for i in right])
                if(not var):
                    new_rules[prod].append(right)
                else:
                    # find non-nullable variables/terminals in production, because it must be present while removing null productions
                    non_nullable = [i for i in right if i not in null_var]

                    c1 = Counter(non_nullable)  # for checking 
                    possible_pairs = ["".join(i) for i in powerset(right)]
                    
                    for subset in possible_pairs:
                        if(subset==''):
                            continue
                        c2 = Counter([i for i in subset if i not in null_var])  # all non-nullable character should not be changed while removing nullable character
                        if(c1==c2 and subset not in new_rules[prod]):
                            new_rules[prod].append(subset)

        self.productions = new_rules
    
    def simplifyCFG(self):
        """
            Remove Useless Productions

            Remove Unit Production
            
            Remove Null Production
        """
        self.remove_useless_production()
        self.remove_unit_production()
        self.remove_useless_production()
        self.remove_null_production()
    
    def isCNF(self):
        
        """ Checks Whether Every Production is in CNF Form or Not : A->BC or A->a """

        for prod in self.productions:
            for right in self.productions[prod]:
                if(right in self.terminals):
                    continue
                if(len(right)!=2): return False
                if(right[0] not in self.variables or right[1] not in self.variables): return False
        return True

    def convertCFG2CNF(self):
        
        """ ALL PRODUCTIONS MUST HAVE ONE OF THE FORMS

            A -> BC or A->a (2 Non terminals or 1 terminal on rhs)
        
            where, A,B,C are Non-Terminals and a is Terminal    
        """
        
        self.simplifyCFG()

        # first assign all terminals to variable
        char = [chr(i+65) for i in range(26)]
        mapping = dict() # for mapping
        
        for terminal in self.terminals:
            for choice in char:
                if choice not in self.variables:
                    self.productions[choice] = [terminal]
                    self.variables.add(choice)
                    mapping[terminal] = choice
                    break
        
        new_rules = dict()

        for prod in self.productions:
            new_rules[prod] = []
            for right in self.productions[prod]:
                if(right in self.terminals):
                    new_rules[prod].append(right)
                    continue
                s = ""
                for j in right:
                    if(j in mapping):
                        s += mapping[j]
                    else:
                        s += j
                new_rules[prod].append(s)
        
        self.productions = new_rules
        self.remove_useless_production()
        
        # continue the loop until CNF form is obtain
        while(not self.isCNF()):
            new_rules = dict()

            # replacing left 2 variables

            for prod in self.productions:
                new_rules[prod] = []
                for right in self.productions[prod]:
                    if(right in self.terminals or len(right) == 2):
                        new_rules[prod].append(right)
                        continue

                    new_var,rem = right[:2],right[2:]
                    for choice in char:
                        if choice not in self.variables:
                            new_rules[choice] = [new_var]
                            self.variables.add(choice)
                            mapping[new_var] = choice
                            break
                    
                    new_rules[prod].append(mapping[new_var]+rem)
                        

                    
            self.productions = new_rules
            self.remove_useless_production()
        
    def is_cfg_empty(self):

        """ CHECKING WHETHER THE LANGUAGE GENERATED BY CFG IS EMPTY OR NON-EMPTY """
        
        # if S is not present in productions or variables, then cfg is empty
        self.simplifyCFG()
        return 'S' not in self.variables
    
    def is_cfg_finite(self)->bool:
        
        """ CHECKING WHETHER THE LANGUAGE GENERATED BY CFG IS FINITE OR INFINTE """
        
        self.convertCFG2CNF()
        # assign number to each variable
        mapVar = {}
        var = list(self.variables)
        for i in range(len(var)):
            mapVar[var[i]] = i
        
        # make a graph
        adj = [[0 for _ in range(len(var))] for _ in range(len(var))]

        for prod in self.productions:
            for right in self.productions[prod]:
                if(len(right)==1):  # must be terminal character
                    continue
                for k in right:
                    if(k.isupper()):
                        adj[mapVar[prod]][mapVar[k]] = 1
        
        n = len(self.variables)

        visited = [False for _ in range(n)]
        pathVis = [False for _ in range(n)]

        def detectCycle(node:int)->bool:
            visited[node] = True
            pathVis[node] = True

            for i in range(n):
                if(adj[node][i] == 1):
                    if(not visited[i]):
                        if(detectCycle(i)):
                            return True
                    elif(pathVis[i]):
                        return True
            
            pathVis[node] = False # backtrack
            return False
        
        return not detectCycle(0)  # if cycle detected, then not finite

In [14]:
def powerset(s):
    if len(s) == 0:
        return [[]]
    else:
        result = []
        for subset in powerset(s[1:]):
            result.append(subset)
            result.append([s[0]]+subset)
        return result
# powerset('aSA')
["".join(i) for i in powerset('aaSA')]

['',
 'a',
 'a',
 'aa',
 'S',
 'aS',
 'aS',
 'aaS',
 'A',
 'aA',
 'aA',
 'aaA',
 'SA',
 'aSA',
 'aSA',
 'aaSA']

In [38]:
a = Counter([])
b = Counter([])
print(a==b)

True


In [5]:
G1 = """
    S->aA|bC
    A->aA|bB|ε
    B->bB/b/ε
    C->cB/cC/ε
    """
G2 = """
    S->AB
    A->BC
    B->CC
    C->b/a
    """
G3 = """
    S->AB
    A->BC
    B->CC
    C->AB/a
    """
G4 = """
    S->CA|BB
    B->b|SB
    C->D
    D->b
    A->a
    E->F
    F->a
    """
G5 = """
    S->AB|AC
    A->aAb|bAa|a
    B->bbA|aaB|AB
    C->abCa|aDb
    D->bD|aC
    E->a
    """
G6 = """
    S->AB
    A->a
    B->C|b
    C->D
    D->E
    E->a
    """
G7 = """
    S->AB
    A->aAA|ε
    B->bBB|ε
    """
G8 = """
    S->aA|aBB
    A->aaA|ε
    B->bB|bbC
    C->B
    D->F
    F->a
    """
G9 = """
    S->AB
    A->aB|b
    B->aC|bB
    C->bB
    D->a
    """
G10 = """
    S->aSb|ab
    """
G11 = """
    S->bA|aB
    A->bAA|aS|a
    B->aBB|bS|b
    """
G12 = """
    S->aAB|Bb
    A->a
    B->b
    """

In [6]:
cfg = CFG(productions=G5)
print(cfg)

CFG (V,T,S,P) = {
	V = {'B', 'A', 'S', 'D', 'C', 'E'}
	T = {'b', 'a'}
	S = 'S'
	P = {'S->AB', 'C->abCa', 'E->a', 'A->bAa', 'B->aaB', 'D->aC', 'A->aAb', 'C->aDb', 'B->bbA', 'B->AB', 'S->AC', 'A->a', 'D->bD'}
}


In [7]:
cfg.productions

{'S': ['AB', 'AC'],
 'A': ['aAb', 'bAa', 'a'],
 'B': ['bbA', 'aaB', 'AB'],
 'C': ['abCa', 'aDb'],
 'D': ['bD', 'aC'],
 'E': ['a']}

### USELESS PRODUCTIONS -> Elimination of useless productions is need to elimate those variables which do not generate any terminal string & eliminate variables which are not required for derivation

In [7]:
cfg.reachableFromStart()

{'A', 'B', 'C', 'D', 'S'}

In [8]:
cfg.removeUnreachable(cfg.reachableFromStart())
print(cfg)

CFG (V,T,S,P) = {
	V = {'D', 'S', 'A', 'C', 'B'}
	T = {'a', 'b'}
	S = 'S'
	P = {'S->AB', 'C->abCa', 'A->bAa', 'A->aAb', 'B->aaB', 'B->AB', 'S->AC', 'C->aDb', 'B->bbA', 'A->a', 'D->aC', 'D->bD'}
}


In [9]:
cfg.deriveTerminalSymbol()

{'A', 'B', 'S'}

In [10]:
cfg.removeProductive(cfg.deriveTerminalSymbol())
print(cfg)

CFG (V,T,S,P) = {
	V = {'A', 'B', 'S'}
	T = {'a', 'b'}
	S = 'S'
	P = {'S->AB', 'A->bAa', 'A->aAb', 'B->aaB', 'B->AB', 'B->bbA', 'A->a'}
}


In [58]:
# or directly
cfg.remove_useless_production()
print(cfg)

CFG (V,T,S,P) = {
	V = {'A', 'S', 'B'}
	T = {'a', 'b'}
	S = 'S'
	P = {'A->a', 'B->aaB', 'B->AB', 'S->AB', 'B->bbA', 'A->bAa', 'A->aAb'}
}


### UNIT PRODUCTION REMOVAL : FORM A -> B (where both are non-terminals) -> these function will remove unit production but will produce useless production, hence need to remove it afterwards

In [11]:
cfg2 = CFG(productions=G6)
print(cfg2)

CFG (V,T,S,P) = {
	V = {'D', 'S', 'A', 'E', 'C', 'B'}
	T = {'a', 'b'}
	S = 'S'
	P = {'S->AB', 'B->C', 'B->b', 'E->a', 'C->D', 'A->a', 'D->E'}
}


In [12]:
cfg2.remove_unit_production()
print(cfg2)

CFG (V,T,S,P) = {
	V = {'D', 'S', 'A', 'E', 'C', 'B'}
	T = {'a', 'b'}
	S = 'S'
	P = {'S->AB', 'B->b', 'D->a', 'E->a', 'C->a', 'A->a', 'B->a'}
}


In [13]:
cfg2.remove_useless_production()
print(cfg2)

CFG (V,T,S,P) = {
	V = {'A', 'S', 'B'}
	T = {'a', 'b'}
	S = 'S'
	P = {'B->a', 'S->AB', 'A->a', 'B->b'}
}


### NULL PRODUCTION REMOVAL : Removes ε from every production

In [72]:
cfg3 = CFG(productions=G7)
print(cfg3)

CFG (V,T,S,P) = {
	V = {'A', 'S', 'B'}
	T = {'ε', 'a', 'b'}
	S = 'S'
	P = {'A->aAA', 'A->ε', 'S->AB', 'B->bBB', 'B->ε'}
}


In [73]:
cfg3.remove_null_production()
print(cfg3)

CFG (V,T,S,P) = {
	V = {'A', 'S', 'B'}
	T = {'ε', 'a', 'b'}
	S = 'S'
	P = {'A->a', 'B->b', 'A->aAA', 'S->AB', 'B->bBB', 'S->A', 'A->aA', 'S->B', 'B->bB'}
}


### COMBINING THEM

In [83]:
cfg4 = CFG(productions=G8)  # contans all useless,unit,null productions
print(cfg4)

CFG (V,T,S,P) = {
	V = {'A', 'C', 'F', 'S', 'D', 'B'}
	T = {'ε', 'a', 'b'}
	S = 'S'
	P = {'D->F', 'A->ε', 'S->aA', 'F->a', 'A->aaA', 'B->bbC', 'C->B', 'B->bB', 'S->aBB'}
}


In [84]:
cfg4.simplifyCFG()
print(cfg4)

CFG (V,T,S,P) = {
	V = {'A', 'S'}
	T = {'ε', 'a', 'b'}
	S = 'S'
	P = {'S->a', 'A->aaA', 'A->aa', 'S->aA'}
}


#### EMPTYNESS PROBLEM : CHECKING WHETHER THE GRAMMER IS EMPTY OR NOT, SIMPLIFY THE GRAMMER, IF STARTING SYMBOL REMOVED, then GRAMMER IS EMPTY

In [93]:
cfg5 = CFG(productions=G9)
print(cfg5)

CFG (V,T,S,P) = {
	V = {'A', 'C', 'S', 'D', 'B'}
	T = {'a', 'b'}
	S = 'S'
	P = {'S->AB', 'A->aB', 'C->bB', 'A->b', 'B->aC', 'D->a', 'B->bB'}
}


In [94]:
cfg5.simplifyCFG()
print(cfg5)

CFG (V,T,S,P) = {
	V = set()
	T = {'a', 'b'}
	S = 'S'
	P = set()
}


In [95]:
cfg5.is_cfg_empty()

True

### CONVERSION OF CFG TO CNF (CHOMASKY NORMAL FORM)

In [52]:
cfg6 = CFG(productions=G11)
print(cfg6)

CFG (V,T,S,P) = {
	V = {'S', 'A', 'B'}
	T = {'a', 'b'}
	S = 'S'
	P = {'A->bAA', 'S->aB', 'B->aBB', 'B->bS', 'A->aS', 'S->bA', 'A->a', 'B->b'}
}


In [53]:
cfg6.isCNF()

False

In [54]:
cfg6.convertCFG2CNF()
cfg6.isCNF()

True

In [55]:
print(cfg6)

CFG (V,T,S,P) = {
	V = {'S', 'A', 'B', 'E', 'C', 'F', 'D'}
	T = {'a', 'b'}
	S = 'S'
	P = {'C->a', 'D->b', 'E->DA', 'S->DA', 'A->EA', 'F->CB', 'S->CB', 'A->CS', 'B->DS', 'A->a', 'B->FB', 'B->b'}
}


#### FINITENESS PROBLEM : CHECKING WHETHER THE LANGUAGE GENERATED BY CFG IS FINITE OR INFINITE

In [57]:
cfg6.is_cfg_finite()

False

In [5]:
cfg7 = CFG(productions=G2)
print(cfg7)
cfg8 = CFG(productions=G3)
print(cfg8)

CFG (V,T,S,P) = {
	V = {'C', 'B', 'S', 'A'}
	T = {'a', 'b'}
	S = 'S'
	P = {'A->BC', 'B->CC', 'C->b', 'S->AB', 'C->a'}
}
CFG (V,T,S,P) = {
	V = {'C', 'B', 'S', 'A'}
	T = {'a'}
	S = 'S'
	P = {'C->AB', 'A->BC', 'B->CC', 'S->AB', 'C->a'}
}


In [6]:
cfg7.is_cfg_finite()

True

In [7]:
cfg8.is_cfg_finite()

False